In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22457822
paper_name = 'chesi_gitler_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/yeast deletions Mn.xlsx', sheet_name='single deletion')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4842 x 19


In [7]:
original_data.head()

,ORF name,0 mM size-1 1,0 mM size-2 1,12 mM size-1 1,12 mM size-2 1,25 mM size-1 1,25 mM size-2 1,0 mM size-1 2,0 mM size-2 2,12 mM size-1 2,12 mM size-2 2,25 mM size-1 2,25 mM size-2 2,0 mM size-1 3,0 mM size-2 3,12 mM size-1 3,12 mM size-2 3,25 mM size-1 3,25 mM size-2 3
0,YAL002W,239,228,257,249,112,110,377,362,214,225,109,111,391,379,212,201,123,114
1,YAL004W,341,300,276,254,66,88,442,414,223,205,78,61,437,406,261,235,69,52
2,YAL005C,250,245,230,221,88,92,356,362,182,186,74,61,356,372,221,226,64,62
3,YAL007C,266,293,205,222,100,104,374,391,213,233,80,88,338,348,244,265,58,63
4,YAL008W,227,237,206,206,93,91,341,336,194,197,69,70,298,290,206,204,52,47


In [8]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF name, 0 mM size-1 1, 0 mM size-2 1, 12 mM size-1 1, 12 mM size-2 1, 25 mM size-1 1, 25 mM size-2 1, 0 mM size-1 2, 0 mM size-2 2, 12 mM size-1 2, 12 mM size-2 2, 25 mM size-1 2, 25 mM size-2 2, 0 mM size-1 3, 0 mM size-2 3, 12 mM size-1 3, 12 mM size-2 3, 25 mM size-1 3, 25 mM size-2 3, orf]
Index: []


In [15]:
original_data.set_index('orf', inplace=True)
original_data.drop(columns=['ORF name'], inplace=True)

In [17]:
cols = original_data.columns.values
conc = [c.split(' ')[0] for c in cols]
original_data.columns = conc

In [18]:
original_data = original_data.astype(float)

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

In [22]:
original_data.shape

(4755, 3)

In [24]:
original_data['12'] = original_data['12'] / original_data['0']
original_data['25'] = original_data['25'] / original_data['0']

In [26]:
original_data.drop(columns=['0'], inplace=True)

# Prepare the final dataset

In [27]:
data = original_data.copy()

In [28]:
dataset_ids = [247,248]
datasets = datasets.reindex(index=dataset_ids)

In [29]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [30]:
data.head()

dataset_id,247,248
data_type,value,value
orf,,
YAL002W,0.687247,0.343623
YAL004W,0.621368,0.176923
YAL005C,0.652241,0.227202
YAL007C,0.687562,0.245274
YAL008W,0.701562,0.244072


## Subset to the genes currently in SGD

In [31]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [32]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,247,248
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.687247,0.343623
1863,YAL004W,0.621368,0.176923
4,YAL005C,0.652241,0.227202
5,YAL007C,0.687562,0.245274
6,YAL008W,0.701562,0.244072


# Normalize

In [33]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [34]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [35]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,247,248,247,248
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.687247,0.343623,0.209066,1.015004
1863,YAL004W,0.621368,0.176923,-0.305109,-0.553236
4,YAL005C,0.652241,0.227202,-0.064147,-0.080231
5,YAL007C,0.687562,0.245274,0.211527,0.089774
6,YAL008W,0.701562,0.244072,0.320789,0.078467


# Print out

In [36]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [37]:
from IO.save_data_to_db3 import *

In [38]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 22457822...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.63s/it]

Updating the data_modified_on field...
